# Optimization of the hyperparameters of a Neural Network with Keras

### Level: Intermediate

In this notebook, we show how to use Keras tuner to optimize ANN hiperparameters. 

First, we create a basic neural network for this task. Then, we adress how to optimize some inner and outer hiperparameters.

At the end of this notebook there is some useful documentation of this topic.

#### Dependencies

In [ ]:
import keras
import sklearn

import keras_tuner as kt
import pandas as pd

#### Data

Load train data and split into trai/test groups.

In [ ]:
X = pd.read_parquet("features.parquet")
y = pd.read_parquet("targets.parquet")

# we set a fixed random state for reproducibility and teaching purposes,
# but our results have to be consistent across multiple seeds to be relevant
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(
    X, y, train_size=0.9, test_size=0.1, random_state=0
)

# shape of input features and output predictions
features_shape = X_train.iloc[0].shape
target_shape = y_train.iloc[0].shape

#### Some parameters

In [ ]:
# model optimizer
loss = keras.losses.MSE()
lr = {"min_value": 1e-4, "max_value": 1e-2}  # values from 0.0001 to 0.01
metrics = "val_loss"

# training
epochs = 200
batch_size = 64
early_stopping_patience = int(0.1 * epochs)

# hp tuner
max_trials = 50

#### Neural Network model

We employ the `Functional API` inside of a function to define a model for hyperparameter search [1], with an input for hyperparameters to search as indicated.

In this example, we just go for the optimization of the learning rate, this is, how much we change the NN parameters each train step.

In [ ]:
def model_builder(hp):

    input_layer = keras.Input(shape=features_shape)
    inner_layer_1 = keras.layers.Dense(64, activation="selu")(input_layer)
    inner_layer_2 = keras.layers.Dense(32, activation="selu")(inner_layer_1)
    inner_layer_3 = keras.layers.Dense(16, activation="selu")(inner_layer_2)
    output_layer = keras.layers.Dense(target_shape)(inner_layer_3)

    model = keras.Model(inputs=input_layer, outputs=output_layer, name="NN_model")

    # Tune the learning rate for the optimizer, choose an optimal value [2]
    hp_learning_rate = hp.Float(
        "learning_rate", min_value=lr["min_value"], max_value=lr["max_value"]
    )

    model.compile(
        loss=loss,
        optimizer=keras.optimizers.Nadam(learning_rate=hp_learning_rate),
        metrics=metrics,
    )

    return model

#### Inner Hyperparameter optimization

Defined the model, select the method to achieve the tuning [3] and perform it.

In [ ]:
# we set a fixed random state for reproducibility and teaching purposes, but our
# results have to be consistent across multiple seeds to be relevant, which can
# be easily implemented avoiding the fixing of a `seed` and setting
# `executions_per_trial` to more than one
tuner = kt.GridSearch(
    hypermodel=model_builder,  # model to tune that takes hyperparameters and returns a Model instance
    objective=metrics,  # direction of the optimization
    max_trials=max_trials,  # total number of model configurations to test
    tune_new_entries=True,  # if hyperparameter entries requested by the hypermodel should be added to the search space
    allow_new_entries=True,
    seed=0,
    project_name="KerasTuner",  # prefix for files saved by this Tuner, which are control points for each model configuration
    # executions_per_trial = 10
)
# NOTE: if hyperparameter search is executed again, Keras Tuner will use
# `project_name` saved files to resume the search. To avoid that, set
# `overwrite=True`.

# set an early stopping for training
callbacks = [
    keras.callbacks.EarlyStopping(monitor=metrics, patience=early_stopping_patience)
]

# finally tune the hyperpararmeters, using the 10 percent of train data for
# validation. Input arguments are the same than those for keras.model.fit [4]
tuner.search(
    X_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.1,
    callbacks=callbacks,
)

# print search space summary.
tuner.search_space_summary(extended=False)

# get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(
    f"""
The hyperparameter search is complete. The optimal learning rate for the
optimizer is {best_hps.get('learning_rate')}.
"""
)

#### Outer Hyperparameter optimization

Found the optimal inner hyperparameters, use them for training and find the optimal outer hyperparameters, such as the number of epochs during training.

This epoch optimization can be safely skipped if, for example, early stopping is considered to be enough to avoid the lost of performance or the saving of the models at different epochs is implemented. [5]

In [ ]:
# build the model with the optimal hyperparameters
model = tuner.hypermodel.build(best_hps)

# train the model again
callbacks = [
    keras.callbacks.EarlyStopping(monitor=metrics, patience=early_stopping_patience)
]
history = model.fit(
    X_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.1,
    callbacks=callbacks,
)

# obtain the epoch with the best loss
val_loss_per_epoch = history.history[metrics]
best_epoch = val_loss_per_epoch.index(min(val_loss_per_epoch)) + 1
print("Best epoch: %d" % (best_epoch,))

#### Last train with optimized hyperparameters

Finally, train the final version of the model with optimized hyperparameters, evaluate and save it.

In [ ]:
# build again the model with the optimal hyperparameters
hypermodel = tuner.hypermodel.build(best_hps)

# retrain the model for the last time
hypermodel.fit(
    X_train,
    y_train,
    batch_size=batch_size,
    epochs=best_epoch,
    validation_split=0.1,
    callbacks=callbacks,
)

score = hypermodel.evaluate(X_test, y_test, verbose=0)
print("[test loss, test accuracy]:", score)

model = keras.saving.load_model("opt_hp_model.keras")

#### References

.. [1] https://www.tensorflow.org/tutorials/keras/keras_tuner?hl=es-419 

.. [2] https://keras.io/api/keras_tuner/hyperparameters/

.. [3] https://keras.io/api/keras_tuner/tuners/

.. [4] https://keras.io/api/keras_tuner/tuners/base_tuner/ 

.. [5] https://keras.io/api/callbacks/model_checkpoint/ 